# Data Cleaning & Merging
**Analyst:** Ahmad

## 1. Data Cleaning

In [1]:
import sys
sys.path.append("../src")

from data.loader import load_excel_sheets, summarize_excel_sheets
from data.quality import plot_missing_heatmaps
# !pip install gender-guesser 

file_path = "../data/processed/Project Data.xlsx"
datasets = load_excel_sheets(file_path)


(1) Product Lookup

🔸 Notes:

There are 134 missing values ​​in ProductColor and ProductStyle.

The rest of the columns are clean.

In [2]:
# Display unique values ​​in the ProductColor column
print(datasets[' Product Lookup']['ProductColor'].value_counts(dropna=False))

ProductColor
Black           88
NaN             50
Red             37
Silver          36
Yellow          36
Blue            26
Multi            8
Silver/Black     7
White            4
Grey             1
Name: count, dtype: int64


In [3]:
# Display rows with NaN values ​​in ProductColor column
nan_rows = datasets[' Product Lookup'][datasets[' Product Lookup']['ProductColor'].isna()]
nan_rows.head(5)

,ProductKey,ProductSubcategoryKey,ProductSKU,ProductName,ModelName,ProductDescription,ProductColor,ProductStyle,ProductCost,ProductPrice,SKU Type,Discount Price
95,391,10,FK-1639,LL Fork,LL Fork,Stout design absorbs shock and offers more pre...,NaN,NaN,65.8097,148.22,FK-1639,133.398
96,392,10,FK-5136,ML Fork,ML Fork,Composite road fork with an aluminum steerer t...,NaN,NaN,77.9176,175.49,FK-5136,157.941
97,393,10,FK-9939,HL Fork,HL Fork,High-performance carbon road fork with curved ...,NaN,NaN,101.8936,229.49,FK-9939,206.541
98,394,11,HS-0296,LL Headset,LL Headset,Threadless headset provides quality at an econ...,NaN,NaN,15.1848,34.20,HS-0296,30.780
99,395,11,HS-2451,ML Headset,ML Headset,Sealed cartridge keeps dirt out.,NaN,NaN,45.4168,102.29,HS-2451,92.061


After searching by product name on Google: 

- Most of the products whose colors are marked with "Null" are a mixture of more than one color, 
- so we can add a new value and call it “mixed.”

In [4]:
# Replace the NaN value in the ProductColor column with the value "Mixed"
datasets[' Product Lookup']['ProductColor'] = datasets[' Product Lookup']['ProductColor'].fillna('Mixed')
# Verify after modification
print(datasets[' Product Lookup']['ProductColor'].value_counts())

ProductColor
Black           88
Mixed           50
Red             37
Silver          36
Yellow          36
Blue            26
Multi            8
Silver/Black     7
White            4
Grey             1
Name: count, dtype: int64


In [5]:
# Display unique values ​​in the ProductStyle column
unique_styles = datasets[' Product Lookup']['ProductStyle'].unique()
print(datasets[' Product Lookup']['ProductStyle'].value_counts(dropna=False))


ProductStyle
U      174
NaN     84
W       28
M        7
Name: count, dtype: int64


In [6]:
# Display rows with NaN values ​​in ProductStyle column
nan_rows = datasets[' Product Lookup'][datasets[' Product Lookup']['ProductStyle'].isna()]
nan_rows.head(5)

,ProductKey,ProductSubcategoryKey,ProductSKU,ProductName,ModelName,ProductDescription,ProductColor,ProductStyle,ProductCost,ProductPrice,SKU Type,Discount Price
0,214,31,HL-U509-R,"Sport-100 Helmet, Red",Sport-100,"Universal fit, well-vented, lightweight , snap...",Red,NaN,13.0863,34.9900,HL-U509,31.49100
1,215,31,HL-U509,"Sport-100 Helmet, Black",Sport-100,"Universal fit, well-vented, lightweight , snap...",Black,NaN,12.0278,33.6442,HL-U509,30.27978
4,220,31,HL-U509-B,"Sport-100 Helmet, Blue",Sport-100,"Universal fit, well-vented, lightweight , snap...",Blue,NaN,12.0278,33.6442,HL-U509,30.27978
95,391,10,FK-1639,LL Fork,LL Fork,Stout design absorbs shock and offers more pre...,Mixed,NaN,65.8097,148.2200,FK-1639,133.39800
96,392,10,FK-5136,ML Fork,ML Fork,Composite road fork with an aluminum steerer t...,Mixed,NaN,77.9176,175.4900,FK-5136,157.94100


A product's style is one of three things.

- M => Men’s
- W => Women’s
- U => Unisex

Most products with a value of "none" are unisex.
After searching on google and view describtion of some products
Most products are suitable for both genders, such as:

- Headphones
- Wheel bar
- Forks

That's why we'll add them to 'U'.

In [7]:
# Replace the NaN value in the ProductStyle column with the value "U"
datasets[' Product Lookup']['ProductStyle'] = datasets[' Product Lookup']['ProductStyle'].fillna('U')
# Verify after modification
print(datasets[' Product Lookup']['ProductStyle'].value_counts())

ProductStyle
U    258
W     28
M      7
Name: count, dtype: int64


(2) Customer Lookup

🔸 Notes:

There are 260 missing values ​​in Prefix and Gender.

Some clients may not have prefix or gender data.

The prefix depends on gender and marital status.

🔹 Required:

Fill in the missing values ​​with logical values:

In [8]:
# Display unique values ​​in the Gender column
print(datasets['Customer Lookup']['Gender'].value_counts(dropna=False))
print(datasets['Customer Lookup']['Prefix'].value_counts(dropna=False))

Gender
M      9126
F      8892
NaN     130
Name: count, dtype: int64
Prefix
Mr.     9126
Mrs.    6422
Ms.     2470
NaN      130
Name: count, dtype: int64


In [9]:
# Display rows with NaN values ​​in Gender and Prefix columns
nan_rows = datasets['Customer Lookup'][datasets['Customer Lookup']['Gender'].isna()]
nan_rows.head(5)

,CustomerKey,Prefix,FirstName,LastName,BirthDate,MaritalStatus,Gender,EmailAddress,AnnualIncome,TotalChildren,EducationLevel,Occupation,HomeOwner,Full Name,Domain Name
24,11025,NaN,Alejandro,Beck,16794,M,NaN,alejandro45@adventure-works.com,10000,2,Partial High School,Clerical,Y,Alejandro Beck,Adventure Works
34,11035,NaN,Wendy,Dominguez,17587,M,NaN,wendy12@adventure-works.com,10000,2,Partial High School,Clerical,Y,Wendy Dominguez,Adventure Works
81,11082,NaN,Angela,Butler,24323,S,NaN,angela17@adventure-works.com,130000,0,Graduate Degree,Management,N,Angela Butler,Adventure Works
228,11234,NaN,Anna,Griffin,19317,S,NaN,anna46@adventure-works.com,70000,4,Partial College,Professional,Y,Anna Griffin,Adventure Works
464,11475,NaN,Cesar,Subram,28772,S,NaN,cesar12@adventure-works.com,30000,0,High School,Manual,N,Cesar Subram,Adventure Works


In [10]:
import gender_guesser.detector as gender
import numpy as np
import pandas as pd

# Create type detector
d = gender.Detector()

# Function to guess gender from name
def guess_gender(name):
    if not isinstance(name, str) or name.strip() == "":
        return np.nan
    g = d.get_gender(name.split()[0])
    if g in ['male', 'mostly_male']:
        return 'M'
    elif g in ['female', 'mostly_female']:
        return 'F'
    else:
        return np.nan

# Update Gender column based on name
datasets['Customer Lookup']['Gender'] = datasets['Customer Lookup'].apply(
    lambda row: guess_gender(row['FirstName']) if pd.isna(row['Gender']) else row['Gender'], axis=1
)

# Function to determine the prefix based on gender and marital status
def assign_prefix(row):
    # If there is an old prefix, leave it as is.
    if pd.notna(row['Prefix']):
        return row['Prefix']

    # Determine the prefix based on gender and marital status
    if row['Gender'] == 'M':
        return 'Mr.'
    elif row['Gender'] == 'F':
        if row['MaritalStatus'] == 'M':  # Married
            return 'Mrs.'
        else:
            return 'Ms.'
    return np.nan

# Update Prefix Column Based on Type
datasets['Customer Lookup']['Prefix'] = datasets['Customer Lookup'].apply(assign_prefix, axis=1)

# Show final results
print("Gender column after filling:")
print(datasets['Customer Lookup']['Gender'].value_counts(dropna=False))
print("\nPrefix column after filling:")
print(datasets['Customer Lookup']['Prefix'].value_counts(dropna=False))


Gender column after filling:
Gender
M      9183
F      8962
NaN       3
Name: count, dtype: int64

Prefix column after filling:
Prefix
Mr.     9183
Mrs.    6459
Ms.     2503
NaN        3
Name: count, dtype: int64


In [11]:
# Display rows with NaN values ​​in Gender column
nan_rows = datasets['Customer Lookup'][datasets['Customer Lookup']['Gender'].isna()]
nan_rows.head(5)

,CustomerKey,Prefix,FirstName,LastName,BirthDate,MaritalStatus,Gender,EmailAddress,AnnualIncome,TotalChildren,EducationLevel,Occupation,HomeOwner,Full Name,Domain Name
6214,17330,NaN,Latoya,Becker,22873,M,NaN,latoya19@adventure-works.com,70000,5,Bachelors,Professional,Y,Latoya Becker,Adventure Works
9265,20448,NaN,Jésus,Blanco,20507,M,NaN,jésus14@adventure-works.com,80000,2,High School,Skilled Manual,Y,Jésus Blanco,Adventure Works
14254,25522,NaN,Casey,Anand,28016,S,NaN,casey22@adventure-works.com,30000,0,Partial College,Clerical,N,Casey Anand,Adventure Works


Three rows remain, which will be filled manually.

In [12]:
gender_map_manual = {
    'Latoya': 'F',
    'Jésus': 'M',
    'Casey': 'M'
}
prefix_map_manual = {
    'Latoya': 'Ms.',
    'Jésus': 'Mr.',
    'Casey': 'Mr.'
}

datasets['Customer Lookup']['Gender'] = datasets['Customer Lookup'].apply(
    lambda row: gender_map_manual.get(row['FirstName'], row['Gender']),
    axis=1
)
datasets['Customer Lookup']['Prefix'] = datasets['Customer Lookup'].apply(
    lambda row: prefix_map_manual.get(row['FirstName'], row['Prefix']),
    axis=1
)
print(datasets['Customer Lookup']['Gender'].value_counts(dropna=False))
print(datasets['Customer Lookup']['Prefix'].value_counts(dropna=False))

Gender
M    9206
F    8942
Name: count, dtype: int64
Prefix
Mr.     9206
Mrs.    6426
Ms.     2516
Name: count, dtype: int64


In [13]:
summary_df = summarize_excel_sheets(datasets)
summary_df

,Sheet,Rows,Columns,Missing Values,Duplicates
0,Product Lookup,293,12,0,0
1,Customer Lookup,18148,15,0,0
2,Product Subcateg,37,3,0,0
3,Product Categori,4,2,0,0
4,Territory Lookup,10,4,0,0
5,Returns Data,1809,4,0,0
6,Sales Data,56046,8,0,0
7,Calendar Lookup,912,9,0,0


Now All data is clean

Let's check the datatypes

(3) Product Subcategory / Category

🔸 No Missing Values.

🔹 But:

The name of the sheet is missing letters:
Product Subcateg → Product Subcategory
Product Categori → Product Category

In [14]:
# Edit the names of the sheets in the dataset
datasets['Product Subcategory'] = datasets.pop('Product Subcateg')
datasets['Product Category'] = datasets.pop('Product Categori')

(4) Returns Data

🔸 No Missing Values.

🔹 But:

Columns: TerritoryKey, ProductKey, ReturnQuantity — very important, you will use them in the merge later.

Make ReturnDate in date format:

In [15]:
datasets['Returns Data']['ReturnDate'] = pd.to_datetime(
    datasets['Returns Data']['ReturnDate'], 
    unit='D', 
    origin='1899-12-30'
)
datasets['Returns Data'].info()
datasets['Returns Data'].head(6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1809 entries, 0 to 1808
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ReturnDate      1809 non-null   datetime64[ns]
 1   TerritoryKey    1809 non-null   int64         
 2   ProductKey      1809 non-null   int64         
 3   ReturnQuantity  1809 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 56.7 KB


,ReturnDate,TerritoryKey,ProductKey,ReturnQuantity
0,2020-01-18,9,312,1
1,2020-01-18,10,310,1
2,2020-01-21,8,346,1
3,2020-01-22,4,311,1
4,2020-02-02,6,312,1
5,2020-02-15,1,312,1


(5) Sales Data

🔸 No Missing Values.

There is no missing, but we need to convert the dates (OrderDate, StockDate) to the format datetime

In [16]:
datasets['Sales Data'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56046 entries, 0 to 56045
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OrderDate      56046 non-null  int64 
 1   StockDate      56046 non-null  int64 
 2   OrderNumber    56046 non-null  object
 3   ProductKey     56046 non-null  int64 
 4   CustomerKey    56046 non-null  int64 
 5   TerritoryKey   56046 non-null  int64 
 6   OrderLineItem  56046 non-null  int64 
 7   OrderQuantity  56046 non-null  int64 
dtypes: int64(7), object(1)
memory usage: 3.4+ MB


In [17]:
datasets['Sales Data']['OrderDate'] = pd.to_datetime(
    datasets['Sales Data']['OrderDate'], 
    unit='D', 
    origin='1899-12-30'
)
datasets['Sales Data']['StockDate'] = pd.to_datetime(
    datasets['Sales Data']['StockDate'], 
    unit='D', 
    origin='1899-12-30'
)
datasets['Sales Data'].info()
datasets['Sales Data'].head(6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56046 entries, 0 to 56045
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   OrderDate      56046 non-null  datetime64[ns]
 1   StockDate      56046 non-null  datetime64[ns]
 2   OrderNumber    56046 non-null  object        
 3   ProductKey     56046 non-null  int64         
 4   CustomerKey    56046 non-null  int64         
 5   TerritoryKey   56046 non-null  int64         
 6   OrderLineItem  56046 non-null  int64         
 7   OrderQuantity  56046 non-null  int64         
dtypes: datetime64[ns](2), int64(5), object(1)
memory usage: 3.4+ MB


,OrderDate,StockDate,OrderNumber,ProductKey,CustomerKey,TerritoryKey,OrderLineItem,OrderQuantity
0,2020-01-01,2019-09-21,SO45080,332,14657,1,1,1
1,2020-01-01,2019-12-05,SO45079,312,29255,4,1,1
2,2020-01-01,2019-10-29,SO45082,350,11455,9,1,1
3,2020-01-01,2019-11-16,SO45081,338,26782,6,1,1
4,2020-01-02,2019-12-15,SO45083,312,14947,10,1,1
5,2020-01-02,2019-10-12,SO45084,310,29143,4,1,1


In [18]:
datasets['Calendar Lookup'].info()
datasets['Calendar Lookup'].head(6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Date              912 non-null    int64 
 1   Day Name          912 non-null    object
 2   Start of Week     912 non-null    int64 
 3   Start of Month    912 non-null    int64 
 4   Start of Quarter  912 non-null    int64 
 5   Month Name        912 non-null    object
 6   Month             912 non-null    int64 
 7   Start of Year     912 non-null    int64 
 8   Year              912 non-null    int64 
dtypes: int64(7), object(2)
memory usage: 64.3+ KB


,Date,Day Name,Start of Week,Start of Month,Start of Quarter,Month Name,Month,Start of Year,Year
0,43831,Wednesday,43829,43831,43831,January,1,43831,2020
1,43832,Thursday,43829,43831,43831,January,1,43831,2020
2,43833,Friday,43829,43831,43831,January,1,43831,2020
3,43834,Saturday,43829,43831,43831,January,1,43831,2020
4,43835,Sunday,43829,43831,43831,January,1,43831,2020
5,43836,Monday,43836,43831,43831,January,1,43831,2020


In [19]:
import pandas as pd

# Columns that need to be converted to dates
date_columns = [
    'Date', 
    'Start of Week', 
    'Start of Month', 
    'Start of Quarter', 
    'Start of Year'
]

# Convert values ​​from Excel numbers to actual dates
for col in date_columns:
    datasets['Calendar Lookup'][col] = pd.to_datetime(
        datasets['Calendar Lookup'][col].astype('float'),
        origin='1899-12-30',
        unit='D',
        errors='coerce'
    )

# Check result
print(datasets['Calendar Lookup'].dtypes)

# Show first 6 rows to check result
print(datasets['Calendar Lookup'].head(6))


Date                datetime64[ns]
Day Name                    object
Start of Week       datetime64[ns]
Start of Month      datetime64[ns]
Start of Quarter    datetime64[ns]
Month Name                  object
Month                        int64
Start of Year       datetime64[ns]
Year                         int64
dtype: object
        Date   Day Name Start of Week Start of Month Start of Quarter  \
0 2020-01-01  Wednesday    2019-12-30     2020-01-01       2020-01-01   
1 2020-01-02   Thursday    2019-12-30     2020-01-01       2020-01-01   
2 2020-01-03     Friday    2019-12-30     2020-01-01       2020-01-01   
3 2020-01-04   Saturday    2019-12-30     2020-01-01       2020-01-01   
4 2020-01-05     Sunday    2019-12-30     2020-01-01       2020-01-01   
5 2020-01-06     Monday    2020-01-06     2020-01-01       2020-01-01   

  Month Name  Month Start of Year  Year  
0    January      1    2020-01-01  2020  
1    January      1    2020-01-01  2020  
2    January      1    2020-01-01

In [20]:
plot_missing_heatmaps(datasets)

## 1. Data Merging

In [21]:
summary_df = summarize_excel_sheets(datasets)

summary_df

,Sheet,Rows,Columns,Missing Values,Duplicates
0,Product Lookup,293,12,0,0
1,Customer Lookup,18148,15,0,0
2,Territory Lookup,10,4,0,0
3,Returns Data,1809,4,0,0
4,Sales Data,56046,8,0,0
5,Calendar Lookup,912,9,0,0
6,Product Subcategory,37,3,0,0
7,Product Category,4,2,0,0


In [22]:
output_path = "../data/processed/Cleaned_data.xlsx"

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    for sheet_name, df in datasets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Data saved successfully to:", output_path)


Data saved successfully to: ../data/processed/Cleaned_data.xlsx
